In [13]:
import pandas as pd
import numpy as np
from joblib import dump
import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

from helpers import MUSIC_GENRES


# Preparación de los datos

Se leen los datos y se aplica el filtrado de outliers utilizando la matriz de covarianza robusta

In [2]:
data_path = '../data/features_3_sec.csv'
df = pd.read_csv(data_path)
df['label'] = df['label'].astype('category')

scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(df.loc[:, 'chroma_stft_mean':'mfcc20_var'].values)
y = df['label'].cat.codes.values

cov = pd.read_csv('../data/kur_covariance_scaled.csv', header=None).values

v = x_scaled - x_scaled.mean(axis=0)
mahalanobis_distances = np.diag(v @ np.linalg.inv(cov) @ v.T)
p25 = np.percentile(mahalanobis_distances, 25)
p75 = np.percentile(mahalanobis_distances, 75)
threshold = p75 + 3 * (p75 - p25)
x_filtered = x_scaled[mahalanobis_distances < threshold]
y_filtered = y[mahalanobis_distances < threshold]

x_train, x_test, y_train, y_test = train_test_split(x_filtered, y_filtered, random_state=42)

# KNN

In [3]:
knn = KNeighborsClassifier(1, p=1, n_jobs=-1).fit(x_train, y_train)
knn_train = knn.score(x_train, y_train)
knn_test = knn.score(x_test, y_test)
print('Score de entrenamiento: ')
print(knn_train)
print('Score de Testeo: ')
print(knn_test)

Score de entrenamiento: 
0.9991865509761388
Score de Testeo: 
0.9459129727531517


In [34]:
y_pred = knn.predict(x_test)
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
print('Precisión: ', precision, ' - Recall: ', recall, ' - f1: ', fscore)
columns = list(MUSIC_GENRES.values())
fig = ff.create_annotated_heatmap(confusion_matrix(y_test, y_pred), x=columns, y=columns, colorscale='viridis')
fig['layout']['yaxis']['autorange'] = "reversed"
fig.write_html('../data/plots/knn_confusion_matrix.html')
fig.show()

Precisión:  0.9463281449533892  - Recall:  0.9459129727531517  - f1:  0.9458828477527101


# Persistencia del modelo

Se guarda el modelo en disco para ser utilizado posteriormente en producción

In [4]:
dump(knn, '../data/models/knn_937.joblib')
dump(scaler, '../data/models/knn_937_scaler.joblib')

['../data/models/knn_937.joblib']

# Support Vector Classifier

In [29]:
svmc = svm.SVC(C=80, gamma=2).fit(x_train, y_train)
svmc_train = svmc.score(x_train, y_train)
svmc_test = svmc.score(x_test, y_test)
print('Score de entrenamiento: ')
print(svmc_train)
print('Score de Testeo: ')
print(svmc_test)

Score de entrenamiento: 
0.9991865509761388
Score de Testeo: 
0.9324928832858885


In [33]:
y_pred = svmc.predict(x_test)
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
print('Precisión: ', precision, ' - Recall: ', recall, ' - f1: ', fscore)
columns = list(MUSIC_GENRES.values())
fig = ff.create_annotated_heatmap(confusion_matrix(y_test, y_pred), x=columns, y=columns, colorscale='viridis')
fig['layout']['yaxis']['autorange'] = "reversed"
fig.write_html('../data/plots/svmc_confusion_matrix.html')
fig.show()

Precisión:  0.9327094893323871  - Recall:  0.9324928832858885  - f1:  0.932481999353169


# Métodos de ensamble - Extra Trees

In [31]:
etc = ExtraTreesClassifier(n_estimators=1000, max_features=57, n_jobs=-1).fit(x_train, y_train)
etc_train = etc.score(x_train, y_train)
etc_test = etc.score(x_test, y_test)
print('Score de entrenamiento: ')
print(etc_train)
print('Score de Testeo: ')
print(etc_test)

Score de entrenamiento: 
0.9991865509761388
Score de Testeo: 
0.906466043106954


In [32]:
y_pred = etc.predict(x_test)
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
print('Precisión: ', precision, ' - Recall: ', recall, ' - f1: ', fscore)
columns = list(MUSIC_GENRES.values())
fig = ff.create_annotated_heatmap(confusion_matrix(y_test, y_pred), x=columns, y=columns, colorscale='viridis')
fig['layout']['yaxis']['autorange'] = "reversed"
fig.write_html('../data/plots/etc_confusion_matrix.html')
fig.show()

Precisión:  0.9089235587116957  - Recall:  0.906466043106954  - f1:  0.9067758655759974
